In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

np.random.seed(0)
number_of_features = 100
features, target = make_classification(n_samples = 10000,
                                       n_features= number_of_features,
                                       n_informative= 3,
                                       n_redundant= 0,
                                       n_classes= 2,
                                       weights = [.5, .5],
                                       random_state= 0)

In [2]:
features

array([[-1.48264759,  0.49409992, -0.47002061, ..., -2.18686734,
        -0.86980592, -1.35467587],
       [ 0.55270454,  0.31498447,  1.30258301, ...,  1.07360917,
         1.50053982,  0.77015201],
       [-0.73794967, -1.42314534,  0.12699839, ...,  1.38149619,
        -0.11916491, -0.22538155],
       ...,
       [ 0.3381805 ,  0.36584927,  1.78666268, ...,  1.43429345,
         0.91819169, -0.71442576],
       [ 0.06045541, -1.58937086,  1.61215731, ...,  0.18918909,
         1.34286687,  1.30312436],
       [-2.43364316,  1.21157165,  1.95942128, ..., -0.18864645,
        -0.02405018,  1.0334714 ]])

In [3]:
target

array([0, 0, 1, ..., 0, 0, 1])

In [4]:
from scikeras.wrappers import KerasClassifier

def create_network():
    network = Sequential()
    network.add(Dense(units= 16, activation='relu', input_shape = (number_of_features,)))
    network.add(Dense(units=16, activation='relu'))
    network.add(Dense(units=1, activation='sigmoid'))
    network.compile(loss='binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])
    return network

neural_network = KerasClassifier(model = create_network, epochs = 10,
                                 batch_size=100, verbose=0)
cross_val_score(neural_network, features, target, cv=3)

array([0.90761848, 0.83258326, 0.89528953])

In [5]:
# 하이퍼파라미터 튜닝 (시간이 오래걸림)

from sklearn.model_selection import GridSearchCV

np.random.seed(0)
number_of_features = 10
features, target = make_classification(n_samples = 100,
                                       n_features= number_of_features,
                                       n_informative= 3,
                                       n_redundant= 0,
                                       n_classes= 2,
                                       weights = [.5, .5],
                                       random_state= 0)

In [6]:
def create_network(optimizer = 'rmsprop'):
    network = Sequential()
    network.add(Dense(units= 16, activation='relu', input_shape = (number_of_features,)))
    network.add(Dense(units=16, activation='relu'))
    network.add(Dense(units=1, activation='sigmoid'))
    network.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy'])
    return network

neural_network = KerasClassifier(model = create_network, verbose=0)
    # ecpoch과 batch는 default값으로 설정되어 있음
epochs = [5,10]
batches = [5, 10, 100]
optimizers = ['rmsprop', 'adam']
hyperparameters = dict(optimizer = optimizers, epochs= epochs, batch_size = batches)

In [7]:
grid = GridSearchCV(estimator= neural_network, param_grid = hyperparameters)
grid_result = grid.fit(features, target)
grid_result.best_params_

{'batch_size': 5, 'epochs': 10, 'optimizer': 'adam'}

In [8]:
grid_result.cv_results_['mean_test_score']

array([0.58, 0.63, 0.65, 0.72, 0.55, 0.53, 0.61, 0.66, 0.53, 0.5 , 0.49,
       0.53])

In [9]:
max(grid_result.cv_results_['mean_test_score'])

0.72

In [10]:
grid_result.best_estimator_

KerasClassifier(
	model=<function create_network at 0x000001CA07BA23A0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=5
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	class_weight=None
)